### Customizing what happens in fit().

When you need to write your own training loop from scratch, you can use the ``GradientTape`` and take control of every little detail.

When you need to customize what ``fit()`` does, you should override the training step function of the Model class. This is the function that is called by ``fit()`` for every batch of data. You will then be able to call ``fit()`` as usual -- and it will be running your own learning algorithm.

* [Docs](https://keras.io/guides/customizing_what_happens_in_fit/)

### Imports

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
import numpy as np

### Configuring the ``device`` for the environment.


In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Let's create a model that will train on the `MNIST` dataset.

In [4]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
X_train.shape

11493376/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

In [5]:
def normalize(image):
  image = tf.convert_to_tensor(image.astype('float32'))/255
  return image

In [6]:
X_train_tensors =tf.convert_to_tensor(list(map(normalize, X_train)))
X_test_tensors = tf.convert_to_tensor(list(map(normalize, X_test)))

y_test_tensors = tf.convert_to_tensor(y_test)
y_train_tensors = tf.convert_to_tensor(y_train)

In [ ]:
y_test_tensors[:2]

<tf.Tensor: shape=(2,), dtype=uint8, numpy=array([7, 2], dtype=uint8)>

 ### Let's create a `Sequential` model that will fit on our custom `.fit()` method.
 We can do this in many ways.

 [2nd way](https://keras.io/guides/customizing_what_happens_in_fit/)

In [70]:
model = keras.Sequential([
      keras.layers.Input(shape=(28, 28,)),
      keras.layers.Flatten(),
      keras.layers.Dense(64, activation="relu"),
      keras.layers.Dense(128, activation="relu"),
      keras.layers.Dense(10, activation="softmax")
])

In [71]:
class CustomFit(keras.Model):
   def __init__(self, model):
    super(CustomFit, self).__init__()
    self.model = model
   def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            # forward pass
            y_pred = self.model(x, training=True)
            # calclate the loss
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # gradient calculations
        gradients = tape.gradient(loss, self.trainable_variables)
        # Update the weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

### Fitting the model using our custom ``fit``.

In [76]:
new_model = CustomFit(model)
new_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                optimizer="adam",
                metrics=["accuracy"]
              )
new_model.fit(X_train_tensors, y_train_tensors, batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0221 - accuracy: 0.9922
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0175 - accuracy: 0.9938
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0175 - accuracy: 0.9941
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0144 - accuracy: 0.9951
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0151 - accuracy: 0.9947


### Now let's create a custom `.fit()` and `.evaluate()`.
We have seen thet to create a `.fit()` method we need to overide the `train_step` which mean if we want to create the `.evaluate()` we need to overide the `test_step`.

In [78]:
class CustomFitEval(keras.Model):
  def __init__(self, model):
    super().__init__()
    self.model = model

  # fit
  def train_step(self, data):
    x, y = data
    # forward pass
    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      # loss
      loss = self.compiled_loss(y, y_pred)
    # caculate the gradients
    gradients = tape.gradient(loss, self.trainable_variables)
    # update the weights
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
      
    # update the metrics
    self.compiled_metrics.update_state(y, y_pred)
    return {m.name: m.result() for m in self.metrics}

  # evaluate
  def test_step(self, data):
    x, y = data
    # making predictions
    y_pred = self.model(x)
    # loss
    loss = self.compiled_loss(y, y_pred)
    self.compiled_metrics.update_state(y, y_pred)
    return {m.name: m.result() for m in self.metrics}

In [81]:
model = keras.Sequential([
      keras.layers.Input(shape=(28, 28,)),
      keras.layers.Flatten(),
      keras.layers.Dense(64, activation="relu"),
      keras.layers.Dense(128, activation="relu"),
      keras.layers.Dense(10, activation="softmax")
])
new_model_1 = CustomFitEval(model)

new_model_1.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                optimizer="adam",
                metrics=["accuracy"]
              )
new_model_1.fit(X_train_tensors, y_train_tensors, batch_size=32, epochs=5,
                validation_data=(X_test_tensors, y_test_tensors), validation_batch_size=16
                )

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2610 - accuracy: 0.9236 - val_loss: 0.1215 - val_accuracy: 0.9629
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1124 - accuracy: 0.9649 - val_loss: 0.0986 - val_accuracy: 0.9709
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0835 - accuracy: 0.9740 - val_loss: 0.0949 - val_accuracy: 0.9701
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0660 - accuracy: 0.9790 - val_loss: 0.0903 - val_accuracy: 0.9718
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0552 - accuracy: 0.9821 - val_loss: 0.0813 - val_accuracy: 0.9761


### Evaluating the `model`

In [80]:
new_model_1.evaluate(X_test_tensors, y_test_tensors, batch_size=32)

313/313 [==============================] - 1s 2ms/step - loss: 0.0886 - accuracy: 0.9736


[0.08860763907432556, 0.9735999703407288]

> That's more of it, in the next notebook we are going to implement our custom `model.compile()`.